In [1]:
# import the MAAP package to handle queries
from maap.maap import MAAP

# import printing package to help display outputs
from pprint import pprint
import s1_rtc

In [2]:
"""
Establish User Inputs for Area of Interest

Arguments
---------
bbox : str    
    Bounding Box for AOI
       "...four latitude and longitude values in the order of [W,S,E,N]"
    Examples:
        Altadena, CA: '-118.253784,34.138953,-118.078002,34.265594'
        Mt. Wilson: '-118.06817369989554,34.22169196645805,-118.05801589904763,34.2282243912438'
dateRange : str
    Date range to query
    Example: '2020-08-31T00:00:00Z,2020-09-16T23:59:59Z'
direction : str
    The satellite's direction of travel; search results will be filtered
    to only include this direction. Must be one of: 'asc', 'dsc', 'both'.
    If 'both', then no filtering will be applied.
"""

# Bounding Box for AOI
bbox = '-118.06817369989554,34.22169196645805,-118.05801589904763,34.2282243912438'  # Mt Wilson (near the Bobcat Fire)

# Date range
dateRange = '2020-08-31T00:00:00Z,2020-09-16T23:59:59Z'  # Bobcat Fire started on Sept 6, 2020 and burned for 103 days.

# Filter for direction of Satellite Travel
direction = 'asc'


In [9]:
# Search for granules
all_results = s1_rtc.get_query_results(bbox, dateRange)
print(f"Total number of search results: {len(all_results)}")


Total number of search results: 6


In [10]:
# Filter for Ascending or Descending only
import importlib; importlib.reload(s1_rtc)

filtered_results = s1_rtc.filter_direction(all_results, direction, verbose=True)
print(f"Total number of search results after filtering for {direction} only: {len(filtered_results)}")


Keeping GranuleID:  S1A_IW_SLC__1SDV_20200908T015033_20200908T015101_034261_03FB47_238D
Keeping GranuleID:  S1B_IW_SLC__1SDV_20200902T014944_20200902T015011_023190_02C091_458A
Keeping GranuleID:  S1B_IW_SLC__1SDV_20200914T014944_20200914T015011_023365_02C60B_B003
Total number of search results after filtering for asc only: 3


In [25]:
print(filtered_results[0].getDownloadUrl())

s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC__1SDV_20200908T015033_20200908T015101_034261_03FB47_238D.zip


In [26]:
url = 's3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC__1SDV_20200908T015033_20200908T015101_034261_03FB47_238D.zip'
filtered_results[0]._convertS3toHttp(url)

'https://asf-ngap2w-p-s1-slc-7b420b89.s3.amazonaws.com/S1A_IW_SLC__1SDV_20200908T015033_20200908T015101_034261_03FB47_238D.zip'

In [27]:
!wget https://asf-ngap2w-p-s1-slc-7b420b89.s3.amazonaws.com/S1A_IW_SLC__1SDV_20200908T015033_20200908T015101_034261_03FB47_238D.zip

--2023-07-10 11:54:19--  https://asf-ngap2w-p-s1-slc-7b420b89.s3.amazonaws.com/S1A_IW_SLC__1SDV_20200908T015033_20200908T015101_034261_03FB47_238D.zip
Resolving asf-ngap2w-p-s1-slc-7b420b89.s3.amazonaws.com (asf-ngap2w-p-s1-slc-7b420b89.s3.amazonaws.com)... 52.92.180.25, 52.92.192.65, 52.218.224.162, ...
Connecting to asf-ngap2w-p-s1-slc-7b420b89.s3.amazonaws.com (asf-ngap2w-p-s1-slc-7b420b89.s3.amazonaws.com)|52.92.180.25|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2023-07-10 11:54:19 ERROR 403: Forbidden.



In [6]:
# WARNING - only use for testing! Not DPS!

# Install all packages for this environment. Send console outputs to a log file.
# We'll simply update the base environment for now.
!mamba env update -n base --file environment.yaml > conda_install_log.txt


In [7]:
# WARNING - only use for testing! Not DPS!
import importlib; importlib.reload(s1_rtc)
import multiprocessing
from multiprocessing.pool import ThreadPool
import os
from itertools import repeat
from time import time

input_dir = "./input"

num_available_cpu = multiprocessing.cpu_count()
print(f"There are {num_available_cpu} CPUs available.")

print(f"Starting a ThreadPool of {5} threads...")
pool = ThreadPool(5)

# Download the S1 SLC granules
start = time()
input_files = pool.starmap(s1_rtc.download_s1_zip, zip(filtered_results, repeat(input_dir)))
print(f"Time to download {len(filtered_results)} S1 SLC granules: {time()-start} seconds")


There are 32 CPUs available.
Starting a ThreadPool of 5 threads...
Time to download 3 S1 SLC granules: 0.005540132522583008 seconds


In [ ]:
# WARNING - only use for testing! Not DPS!
import importlib; importlib.reload(s1_rtc)
from time import time

# Download corresponding files and process through RTC

start = time()
pool.map(s1_rtc.granule2bursts, input_files)
print(f"Time to process {len(input_files)} S1 SLC granules via RTC: {time()-start} seconds")


In [8]:
# # WARNING - only use for testing! Not DPS!
# import importlib; importlib.reload(s1_rtc)
# from time import time

# # Download corresponding files and process through RTC

# start = time()
# # build the command line strings
# !mkdir output

# s1_zip_files = []
# for s1_zip in input_files:
#     s1_zip_files.append(f"python ./s1_rtc.py --s1_zip {s1_zip} -o output")

# pool.map(os.system, s1_zip_files)

# print(f"Time to process {len(input_files)} S1 SLC granules via RTC: {time()-start} seconds")


[07/09 15:22:09] [INFO dem.py] Bounds: -119.40515449012567 33.135928647502695 -115.65862318165676 35.70043235249731
[07/09 15:22:09] [INFO cop_dem.py] Creating scratch/S1B_IW_SLC__1SDV_20200914T014944_20200914T015011_023365_02C60B_B003/dem.tif
[07/09 15:22:09] [INFO cop_dem.py] Fetching remote tiles...
[07/09 15:22:09] [INFO cop_dem.py] Running GDAL command:
[07/09 15:22:09] [INFO cop_dem.py] gdalwarp /vsicurl/https://raw.githubusercontent.com/scottstanie/sardem/master/sardem/data/cop_global.vrt scratch/S1B_IW_SLC__1SDV_20200914T014944_20200914T015011_023365_02C60B_B003/dem.tif -of GTiff -ot Int16 -te -119.405154490125668 33.1359286475026948 -115.658623181656765 35.7004323524973088 -tr 0.000277777777777777778 0.000277777777777777778 -s_srs "epsg:4326+3855" -t_srs "epsg:4326" -wo NUM_THREADS=4 -r nearest -wm 5000 -multi
[07/09 15:22:09] [INFO dem.py] Bounds: -119.52377781121771 33.57918864750269 -115.77355851847517 36.194397352497305
[07/09 15:22:09] [INFO cop_dem.py] Creating scratch/S

Creating output file that is 13501P x 9415L.
0Creating output file that is 13488P x 9232L.
0

[07/09 15:22:09] [INFO dem.py] Bounds: -119.40410139351114 33.1359026475027 -115.65762387163315 35.70026435249731
[07/09 15:22:09] [INFO cop_dem.py] Creating scratch/S1B_IW_SLC__1SDV_20200902T014944_20200902T015011_023190_02C091_458A/dem.tif
[07/09 15:22:09] [INFO cop_dem.py] Fetching remote tiles...
[07/09 15:22:09] [INFO cop_dem.py] Running GDAL command:
[07/09 15:22:09] [INFO cop_dem.py] gdalwarp /vsicurl/https://raw.githubusercontent.com/scottstanie/sardem/master/sardem/data/cop_global.vrt scratch/S1B_IW_SLC__1SDV_20200902T014944_20200902T015011_023190_02C091_458A/dem.tif -of GTiff -ot Int16 -te -119.404101393511141 33.1359026475026965 -115.657623871633149 35.7002643524973067 -tr 0.000277777777777777778 0.000277777777777777778 -s_srs "epsg:4326+3855" -t_srs "epsg:4326" -wo NUM_THREADS=4 -r nearest -wm 5000 -multi


Creating output file that is 13487P x 9232L.
0...10...20...30...40...50...60...70...80...90......10...20...30...40...50...60...70...100 - done.
80...90...100 - done.


OPERA RTC-S1 Science Application Software (SAS) v0.4
Identification:
    processing type: NOMINAL
    product version: 0.4
Ancillary input(s):
    DEM file: scratch/S1B_IW_SLC__1SDV_20200914T014944_20200914T015011_023365_02C60B_B003/dem.tif
Processing parameters:
    apply RTC: True
    apply thermal noise correction: True
    apply absolute radiometric correction: True
    apply valid samples sub-swath masking: True
    apply shadow masking: True
    skip if already processed: False
    scratch dir: scratch/S1B_IW_SLC__1SDV_20200914T014944_20200914T015011_023365_02C60B_B003/temp_1688941406.3642724
    output dir: output/S1B_IW_SLC__1SDV_20200914T014944_20200914T015011_023365_02C60B_B003
    save bursts: True
    save mosaics: False
    save browse: True
    output imagery format: COG
    output imagery compression: DEFLATE
    output imagery nbits: 32
    save secondary layers as HDF5 files: False
    check ancillary coverage: True
Browse images:
    burst height: 1024
    burst width

...

[StdErr] Traceback (most recent call last):
[StdErr]   File "/opt/conda/bin/rtc_s1_single_job.py", line 35, in <module>
[StdErr]     run_single_job(cfg)
[StdErr]   File "/opt/conda/lib/python3.10/site-packages/rtc/rtc_s1_single_job.py", line 1293, in run_single_job
[StdErr]     slantrange_layover_shadow_mask_raster = compute_layover_shadow_mask(
[StdErr]   File "/opt/conda/lib/python3.10/site-packages/rtc/rtc_s1_single_job.py", line 741, in compute_layover_shadow_mask
[StdErr]     gdal_ds = gdal.Open(path_layover_shadow_mask_file,
[StdErr]   File "/opt/conda/lib/python3.10/site-packages/osgeo/gdal.py", line 5297, in Open
[StdErr]     return _gdal.Open(*args)
[StdErr] RuntimeError: scratch/S1B_IW_SLC__1SDV_20200902T014944_20200902T015011_023190_02C091_458A/temp_1688941407.9140255/temp_1688941474.9236443/t064_135516_iw3/layover_shadow_mask_slant_range.tif: No such file or directory
[StdErr] Process ForkPoolWorker-19:
[StdErr] Traceback (most recent call last):
[StdErr]   File "/opt/conda

journal:
 -- DEM EPSG: 4326
 -- Output EPSG: 4326
journal:
 -- Processing block: 0 
 --   - line start: 0
 --   - line end  : 176
 --   - dopplers near mid far: 0 0 0 
journal:
 -- Actual DEM bounds used:
 -- Top Left: -118.028 34.0429
 -- Bottom Right: -116.56 33.391 
 -- Spacing: 0.000277778 -0.000277778
 -- Dimensions: 5289 2348
journal: Total convergence: 78094 out of 88880
journal:
 -- DEM EPSG: 4326
 -- Output EPSG: 4326
journal:
 -- Processing block: 0 
 --   - line start: 0
 --   - line end  : 1000
 --   - dopplers near mid far: 0 0 0 
journal:
 -- Actual DEM bounds used:
 -- Top Left: -118.015 33.9818
 -- Bottom Right: -116.56 33.391 
 -- Spacing: 0.000277778 -0.000277778
 -- Dimensions: 5240 2128
journal:
 -- Processing block: 1 
 --   - line start: 1000
 --   - line end  : 1509
 --   - dopplers near mid far: 0 0 0 
journal:
 -- Actual DEM bounds used:
 -- Top Left: -118.029 34.0448
 -- Bottom Right: -116.585 33.5148 
 -- Spacing: 0.000277778 -0.000277778
 -- Dimensions: 5198

[StdErr] Traceback (most recent call last):
[StdErr]   File "/opt/conda/bin/rtc_s1_single_job.py", line 35, in <module>
[StdErr]     run_single_job(cfg)
[StdErr]   File "/opt/conda/lib/python3.10/site-packages/rtc/rtc_s1_single_job.py", line 1293, in run_single_job
[StdErr]     slantrange_layover_shadow_mask_raster = compute_layover_shadow_mask(
[StdErr]   File "/opt/conda/lib/python3.10/site-packages/rtc/rtc_s1_single_job.py", line 741, in compute_layover_shadow_mask
[StdErr]     gdal_ds = gdal.Open(path_layover_shadow_mask_file,
[StdErr]   File "/opt/conda/lib/python3.10/site-packages/osgeo/gdal.py", line 5297, in Open
[StdErr]     return _gdal.Open(*args)
[StdErr] RuntimeError: scratch/S1B_IW_SLC__1SDV_20200902T014944_20200902T015011_023190_02C091_458A/temp_1688941407.9140255/temp_1688941474.500899/t064_135516_iw2/layover_shadow_mask_slant_range.tif: No such file or directory
[StdErr] Process ForkPoolWorker-10:
[StdErr] Traceback (most recent call last):
[StdErr]   File "/opt/conda/

journal:
 -- DEM EPSG: 4326
 -- Output EPSG: 4326
journal:
 -- Processing block: 0 
 --   - line start: 0
 --   - line end  : 176
 --   - dopplers near mid far: 0 0 0 
journal:
 -- Actual DEM bounds used:
 -- Top Left: -118.176 34.7068
 -- Bottom Right: -116.698 34.0551 
 -- Spacing: 0.000277778 -0.000277778
 -- Dimensions: 5323 2347
journal: Total convergence: 70350 out of 88880
journal:
 -- DEM EPSG: 4326
 -- Output EPSG: 4326
journal:
 -- Processing block: 0 
 --   - line start: 0
 --   - line end  : 1000
 --   - dopplers near mid far: 0 0 0 
journal:
 -- Actual DEM bounds used:
 -- Top Left: -118.162 34.6457
 -- Bottom Right: -116.698 34.0551 
 -- Spacing: 0.000277778 -0.000277778
 -- Dimensions: 5274 2127
journal:
 -- Processing block: 1 
 --   - line start: 1000
 --   - line end  : 1509
 --   - dopplers near mid far: 0 0 0 
journal:
 -- Actual DEM bounds used:
 -- Top Left: -118.176 34.7085
 -- Bottom Right: -116.723 34.179 
 -- Spacing: 0.000277778 -0.000277778
 -- Dimensions: 5

[StdErr] Traceback (most recent call last):
[StdErr]   File "/opt/conda/bin/rtc_s1_single_job.py", line 35, in <module>
[StdErr]     run_single_job(cfg)
[StdErr]   File "/opt/conda/lib/python3.10/site-packages/rtc/rtc_s1_single_job.py", line 1293, in run_single_job
[StdErr]     slantrange_layover_shadow_mask_raster = compute_layover_shadow_mask(
[StdErr]   File "/opt/conda/lib/python3.10/site-packages/rtc/rtc_s1_single_job.py", line 741, in compute_layover_shadow_mask
[StdErr]     gdal_ds = gdal.Open(path_layover_shadow_mask_file,
[StdErr]   File "/opt/conda/lib/python3.10/site-packages/osgeo/gdal.py", line 5297, in Open
[StdErr]     return _gdal.Open(*args)
[StdErr] RuntimeError: scratch/S1B_IW_SLC__1SDV_20200902T014944_20200902T015011_023190_02C091_458A/temp_1688941407.9140255/temp_1688941474.3345978/t064_135520_iw2/layover_shadow_mask_slant_range.tif: No such file or directory
[StdErr] Process ForkPoolWorker-14:
[StdErr] Traceback (most recent call last):
[StdErr]   File "/opt/conda

In [22]:
# Submit DPS Job
maap = MAAP(maap_host='api.dit.maap-project.org')
maap = MAAP(maap_host="api.ops.maap-project.org")
maap = MAAP()

job = []

for item in filtered_results:
    job.append(maap.submitJob(identifier="s1_rtc_mtwilson001",
                   algo_id="S1-RTC-v0.0.1",
                   version="develop",
                   username="niemoell",
                   queue="maap-dps-worker-32gb",
                   in_file=item.getDownloadUrl()
                  )
              )


In [16]:
job[0]

{'job_id': '88ebf44f-b369-4136-a12a-d063d40316d8', 'status': 'Running', 'machine_type': None, 'architecture': None, 'machine_memory_size': None, 'directory_size': None, 'operating_system': None, 'job_start_time': None, 'job_end_time': None, 'job_duration_seconds': None, 'cpu_usage': None, 'cache_usage': None, 'mem_usage': None, 'max_mem_usage': None, 'swap_usage': None, 'read_io_stats': None, 'write_io_stats': None, 'sync_io_stats': None, 'async_io_stats': None, 'total_io_stats': None, 'error_details': None, 'response_code': 200, 'outputs': []}

In [19]:
job[0].retrieve_attributes()
job[0]

{'job_id': '88ebf44f-b369-4136-a12a-d063d40316d8', 'status': 'Failed', 'machine_type': None, 'architecture': None, 'machine_memory_size': None, 'directory_size': None, 'operating_system': None, 'job_start_time': None, 'job_end_time': None, 'job_duration_seconds': None, 'cpu_usage': None, 'cache_usage': None, 'mem_usage': None, 'max_mem_usage': None, 'swap_usage': None, 'read_io_stats': None, 'write_io_stats': None, 'sync_io_stats': None, 'async_io_stats': None, 'total_io_stats': None, 'error_details': None, 'response_code': 200, 'outputs': ['http://maap-ops-workspace.s3-website-us-west-2.amazonaws.com/dataset/triaged_job/S1-RTC-v0.0.1/develop/2023/07/10/88ebf44f-b369-4136-a12a-d063d40316d8', 's3://s3-us-west-2.amazonaws.com:80/maap-ops-workspace/dataset/triaged_job/S1-RTC-v0.0.1/develop/2023/07/10/88ebf44f-b369-4136-a12a-d063d40316d8', 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/dataset/triaged_job/S1-RTC-v0.0.1/develop/2023/07/10/88ebf44f-b369-4136-a12a-d063d40316

In [ ]:
# Collect results and stack into vrt files

# Get list of bursts to stack into a vrt
import glob
import os

# list_of_gran = glob.glob("/projects/rtc_stacks/outputs/*")

list_of_bursts = []
for gran in list_of_gran:
    tmp = glob.glob(os.path.join(gran, "output_dir", "t064_135520_iw1", "*.tif"))
    if tmp:
        list_of_bursts.append(tmp[0])

print(f"{len(list_of_bursts)} total bursts for timeseries.\n{list_of_bursts}")

# Create a stacked .vrt of the timeseries data
#  -separate : since we're using a timeseries for one location, this
#              stacks the .tif files on top of each other
#  -tap      : (target aligned pixels) align the coordinates of the extent 
#              of the output file to the values of the -tr, such that the 
#              aligned extent includes the minimum extent

# !gdalbuildvrt {vrt_file} -separate -tap -tr 25 25 {geotiff_dir}/*.tif*
# !gdalbuildvrt {vrt_file} -separate -overwrite {geotiff_dir}/*.tif*

vrt_inputs_file = "/projects/rtc_stacks/vrt_t064_135520_iw1_inputs.txt"

with open(vrt_inputs_file, "w") as vrt_inputs:
    for burst in list_of_bursts:
        vrt_inputs.write(f"{burst}\n")

!gdalbuildvrt {vrt_file} -separate -overwrite -input_file_list {vrt_inputs_file}